In [1]:
import sys
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error as mae
from rdm_helpers import nse

In [2]:
class ModelCluster:
    INAPPROPRIATE_LEN="The number of parameters passed does not correspond with a number of models"
    
    def __init__(self,n=-1,moedls=[]):
        if n==-1:
            self.models=models
        else:                
            self.models=[]
            for i in range(0,n):
                self.models.append(lgb.LGBMRegressor())
    
    def set_params(self,parameters):
        if len(parameters)==len(self.models):
            for i in range(0,len(self.models)):
                self.models[i].set_params(**(parameters[i]))
        else:
            print(self.INAPPROPRIATE_LEN,file=sys.stderr)
    
    def fit(self,x_train,y_train,x_test,y_test,labels):
        for model,label in zip(self.models,labels):
            model.fit(x_train,y_train[label],eval_set=[(x_test,y_test[label])],verbose=True)
    
    def predict(self,x_test):
        predictions=np.empty((0,len(x_test)),float)
        for model in self.models:
            predictions=np.concatenate((predictions,np.array([model.predict(x_test)])))
        return predictions.T
        

# Testing 

In [3]:
PATH_TO_DATA="data/hydro"
TEST_CSV_NAME='05001_daily'

In [4]:
df=pd.read_csv('%s/%s.csv'%(PATH_TO_DATA,TEST_CSV_NAME),index_col='date')
labels=[]
for i in range(1,11):
    ind='%s%d'%("stage_max",i)
    df[ind]=df['stage_max'].shift(-i)
    labels.append(ind)
to_drop=['stage_max']+labels
df.index=pd.to_datetime(df.index)

In [5]:
modelCluster=ModelCluster(10)

In [6]:
random_state = 228
parameters1 = {     
#     'task': 'train',
#     'boosting_type': 'gbdt',
     'objective': 'rmse',
     'random_state': 42,
     'metric': ['l2', 'mae', 'mse'],
     'drop_rate':0.5,
     'learning_rate': 0.004,
     'feature_fraction': 0.8,
     'bagging_fraction': 0.4,
#     'bagging_freq': 10,
     "n_estimators":1000,
     "max_depth": 8,
     "num_leaves": 32,  
     "max_bin": 32,
     "n_jobs": 4,
     "random_state": random_state,
     "early_stopping_rounds":100,
    
}
parameters2 = {     
#     'task': 'train',
#     'boosting_type': 'gbdt',
     'objective': 'rmse',
     'random_state': 42,
     'metric': ['l2', 'mae', 'mse'],
     'drop_rate':0.5,
     'learning_rate': 0.005,
     'feature_fraction': 0.8,
     'bagging_fraction': 0.4,
#     'bagging_freq': 10,
     "n_estimators":1000,
     "max_depth": 8,
     "num_leaves": 32,  
     "max_bin": 32,
     "n_jobs": 4,
     "random_state": random_state,
     "early_stopping_rounds":100,
    
}
parameters=[parameters1,parameters2]*5

In [7]:
modelCluster.set_params(parameters)

In [8]:
train_end = '2016-12-31'
test_start = '2017-01-01'
test_end = '2017-12-15'
train_start = '2014-12-31'

In [9]:
y=df[labels]
x=df.drop(to_drop,axis=1)
x_train=x[train_start:train_end]
x_test=x[test_start:test_end]
y_train=y[train_start:train_end]
y_test=y[test_start:test_end]

In [10]:
modelCluster.fit(x_train,y_train,x_test,y_test,labels)

C:\Users\taras\Anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 13577.7	valid_0's l1: 88.1237
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 13472.3	valid_0's l1: 87.7847
[3]	valid_0's l2: 13367.6	valid_0's l1: 87.4459
[4]	valid_0's l2: 13264	valid_0's l1: 87.1096
[5]	valid_0's l2: 13161.2	valid_0's l1: 86.7753
[6]	valid_0's l2: 13059	valid_0's l1: 86.4406
[7]	valid_0's l2: 12957.8	valid_0's l1: 86.109
[8]	valid_0's l2: 12857.3	valid_0's l1: 85.7777
[9]	valid_0's l2: 12757.6	valid_0's l1: 85.447
[10]	valid_0's l2: 12658.9	valid_0's l1: 85.1194
[11]	valid_0's l2: 12560.9	valid_0's l1: 84.7922
[12]	valid_0's l2: 12463.5	valid_0's l1: 84.4661
[13]	valid_0's l2: 12366.8	valid_0's l1: 84.1393
[14]	valid_0's l2: 12271	valid_0's l1: 83.8156
[15]	valid_0's l2: 12176	valid_0's l1: 83.4932
[16]	valid_0's l2: 12081.6	valid_0's l1: 83.1708
[17]	valid_0's l2: 11988.3	valid_0's l1: 82.8529
[18]	valid_0's l2: 11895.6	valid_0's l1: 82.535
[19]	valid_0's l2: 11803.4	valid_0's l1: 82.2165
[20]	valid_0's l2: 11712.2	

In [11]:
p=modelCluster.predict(x_test)

In [12]:
mae(y_test,p)

45.55895051499687

In [13]:
np.isnan(y_test.values)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
df

,stage_avg,stage_min,stage_max,temp,water_code,station_id,stage_max1,stage_max2,stage_max3,stage_max4,stage_max5,stage_max6,stage_max7,stage_max8,stage_max9,stage_max10
date,,,,,,,,,,,,,,,,
1984-01-01,258.0,258.0,258.0,NaN,NaN,5001,255.0,252.0,248.0,244.0,244.0,244.0,243.0,242.0,241.0,239.0
1984-01-02,255.0,255.0,255.0,NaN,NaN,5001,252.0,248.0,244.0,244.0,244.0,243.0,242.0,241.0,239.0,237.0
1984-01-03,252.0,252.0,252.0,NaN,NaN,5001,248.0,244.0,244.0,244.0,243.0,242.0,241.0,239.0,237.0,235.0
1984-01-04,248.0,248.0,248.0,NaN,NaN,5001,244.0,244.0,244.0,243.0,242.0,241.0,239.0,237.0,235.0,233.0
1984-01-05,244.0,244.0,244.0,NaN,NaN,5001,244.0,244.0,243.0,242.0,241.0,239.0,237.0,235.0,233.0,231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,399.0,399.0,399.0,NaN,46.0,5001,389.0,384.0,379.0,374.0,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-28,389.0,389.0,389.0,NaN,46.0,5001,384.0,379.0,374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-29,384.0,384.0,384.0,NaN,46.0,5001,379.0,374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [981]:
nse(y_test.values,p)

array([0.99188203, 0.9800937 , 0.96671422, 0.95362772, 0.94044625,
       0.92682945, 0.91528148, 0.90671   , 0.90002458, 0.89694127])